##### ==============================================
##### Marcos Reyes
##### Medellín-Colombia
##### 23/08/2022
##### ==============================================

In [1]:
import sys
sys.path.append('..')

from process_receipt.read_file import check_and_read
from process_receipt.process import get_main_blocks, get_all_line_items, get_company_info, get_total, get_date

In [2]:
file_path = '../test_files/OCR1.txt'
list_all_data, list_only_text = check_and_read(file_path)

In [3]:
blocks = get_main_blocks(list_only_text, list_only_text)

In [4]:
get_all_line_items(blocks[1])

[['8970669', '1', '19.00', '19.00'],
 ['9066468', '1', '8.02', '8.02'],
 ['9557031100236', '1', '3.02', '3.02'],
 ['6935818350846', '1', '3.88', '3.88']]

In [5]:
name, address = get_company_info(blocks[0])
[name, address]

['MR D.T.Y. (JOHOR) SDN BHD',
 'LOT 1851-A & 1851-B, JALAN KPB 6, KAWASAN PERINDUSTRIAN BALAKONG, 43300 SERI KEMBANGAN, SELANGOR (MR DIY TESCO TERBAU)']

In [6]:
get_total(blocks[2])

'33.90'

In [7]:
get_date(blocks[2])

'12-01-19'

## Build Classes

In [8]:
class Item:
    def __init__(self, properties: list) -> None:
        self.sku: str = properties[0]
        self.quantity: str = properties[1]
        self.price: str = properties[2]
        self.total: str = properties[3]

    def __repr__(self) -> str:
        return str(vars(self))
    
    def result(self)-> dict:
        return vars(self)


In [9]:
class Receipt:
    __list_only_text: list = []
    __list_all_data: list = []
    __main_blocks: list = []

    company: str = ''
    date: str = ''
    address: str = ''
    line_items: list = []
    total: str = []

    def __init__(self, file_path: str) -> None:
        self.__list_all_data, self.__list_only_text = check_and_read(file_path)
        self.__main_blocks = get_main_blocks(self.__list_only_text, self.__list_only_text)
        self.company, self.address = get_company_info(self.__main_blocks[0])
        self.total = get_total(self.__main_blocks[2])
        self.date = get_date(self.__main_blocks[2])
        self.line_items = self.__fill_out_line_items(get_all_line_items(self.__main_blocks[1]))

    @staticmethod
    def __fill_out_line_items(list_items: list)-> list:
        items = []
        for item in list_items:
            items.append(Item(item))

        return items

    def __repr__(self) -> str:
        final = {}
        for key, value in vars(self).items():
            if not '__' in key:
                final[key] = value

        return str(final)

    def __getitem__(self, index)-> str:
        return self.line_items[index]

    def get_all_list_info_blocks(self)-> tuple:
        return get_main_blocks(self.__list_only_text, self.__list_all_data)

    def get_min_max(self)-> tuple:
        max_pixel_y = 0
        max_pixel_x = 0

        min_pixel_x = 0
        min_pixel_y = 0

        for list_elements in self.__list_all_data:
            max_pixel_x = max([list_elements[4], max_pixel_x])
            max_pixel_y = max([list_elements[5], max_pixel_y])

            min_pixel_x = min([list_elements[0], min_pixel_x])
            min_pixel_y = min([list_elements[1], min_pixel_x])

        return (max_pixel_x, max_pixel_y, min_pixel_x, min_pixel_y)




In [10]:
item = Item(get_all_line_items(blocks[1])[0])
item.result()

{'sku': '8970669', 'quantity': '1', 'price': '19.00', 'total': '19.00'}

In [11]:
Receipt(file_path).get_all_list_info_blocks()[0]
Receipt(file_path).get_min_max()

(436, 899, 0, 0)